# Persistence

We're going to discuss general IO, how hard disks work, and finally how to design a robust file system.

# Intro to I/O Devices

Computers are designed as a hierarchy of devices which will have different speeds.

CPU/Mem Bus is the fastest.

Below this are busses that connect to other devices:

- Graphics card
- Hard Disk
- Output device (monitors)
- Bluetooth card
- keyboard mouse, USB I/O
- RGB lights
- fans

Accessing any IO device (except maybe the graphics card) is atleast an order of magnitude slower than executing instructions on the CPU.

Many I/O devices have their own microprocessor of dedicated mini CPU so that they can execute independently from the main CPU.

This allows the OS to delegate the work of performing the actual IO to devices.

The OS makes requests of the hard disk.

These requests are made through outward facing registers and a memory buffer:
- COMMAND
- STATUS
- DATA

The OS likes to write the disk is 4KB chunks.

To write to some file, the OS will write the "write" command to COMMAND, then it will copy the 4KB to be written from memory to DATA.

Then the hard disk will write the data to the disk.

STATUS is used by the hard disk to post its status (busy, error statuses, etc).

